In [1]:
import pandas as pd
import csv
import sys
import requests
import math

In [2]:
# original_df = pd.read_csv('ecd/ecd_master_list_original.csv', sep=";", low_memory=False)
original_df = pd.read_csv('ecd/kzn_nw_ilifa.csv', sep=";", low_memory=False)
# Get info on local municipalities
lm_info_df = pd.read_csv('ecd/local_muni_info.csv', sep=";", low_memory=False)

In [3]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3128 entries, 0 to 3127
Columns: 526 entries, PROVINCE to municipality_id
dtypes: float64(254), int64(108), object(164)
memory usage: 12.6+ MB


In [4]:
remove_text = {
    'metropolitan municipality': '',
    'local municipality': '',
}

replace_with = {
    'nelson mandela': 'nelson mandela bay',
    'sol plaatje': 'sol plaatjie',
    'khara hais': '//khara hais',
    'solplatjie': 'sol plaatjie',
    'rustenberg': 'rustenburg',
    'saldanha': 'saldanha bay',
    'plettenberg bay': 'bitou'
}

def clean_muni_name(local_muni):
    if not type(local_muni) is str:
        return local_muni
    
    name = local_muni.lower()

    # Check for some specific cases
    if name in replace_with:
        return replace_with[name]

    # Remove certain phrases
    for k, v in remove_text.iteritems():
        name = name.replace(k, v)

    return name.strip()

In [5]:
df = original_df
# df['local_muni_clean'] = map(clean_muni_name, df["local_municipality"])
df['local_muni_clean'] = map(clean_muni_name, df["LOCAL MUNICIPALITY"])
# df[['address_ward_number']] = df[['address_ward_number']].astype(str)

In [6]:
df = pd.merge(df, lm_info_df, how='left', on="local_muni_clean")

In [7]:
def get_mapit_geo_codes(lat_long):
    latitude, longitude = lat_long.split(",")
    resp = requests.get(
        "http://mapit.code4sa.org/point/4326/%s,%s.json" % (longitude.strip(), latitude.strip()))
    if not resp.json():
        print "3: None"
        return None
    geo_levels = {}
    for item in resp.json().itervalues():
        geo_levels[item['type_name']] = item['codes']['MDB']
    print "3: %s" % (geo_levels.get('Ward'))
    return geo_levels.get('Ward')

In [8]:
def clean_ward_num(ward_num, ward_prefix, gps_location):
    if ward_num:
        if not math.isnan(ward_num):
            ward_num = '%.f' % ward_num
            if len(ward_num) == 8:
                print "1: %s" % (ward_num)
                return ward_num
            if len(ward_num) <= 3:
                if ward_prefix and len(ward_prefix) == 5:
                    ward_prefix = '%.f' % ward_prefix
                    print "2: %s" % (ward_prefix + ('0' * (3-len(ward_num))) + ward_num)
                    return ward_prefix + ('0' * (3-len(ward_num))) + ward_num
    if not type(gps_location) is float:
        return get_mapit_geo_codes(gps_location)
    return None

In [57]:
# geo_cols = ['address_ward_number', 'geo_local_muni_ward_prefix', 'gps_location']
new_df = df
new_df['geo_ward_num_clean'] = map(
    clean_ward_num,
    new_df['address_ward_number'],
    new_df['geo_local_muni_ward_prefix'],
    new_df['gps_location'])

In [10]:
new_df.to_csv('kzn_nw_cleaned_wards.csv', index=False)

In [ ]:
# The code below is to fix some empty geo information

In [183]:
df = pd.read_csv('master_list_cleaned_wards.csv', sep=",", low_memory=False)


In [204]:
def get_local_muni_name(ward_num):
    resp = requests.get(
        "http://mapit.code4sa.org/area/MDB:%s/covered.json" % (ward_num))
    if not resp.json():
        return None
    for level in resp.json().itervalues():
        if level['type_name'] == 'Municipality':
            print level['name']
            return level['name'].lower()

In [198]:
wn = 'na123'
wn.find('z')

-1

In [192]:
df.head(n=2)

,province,ID,date,start,end,started,complete,signature,signature_interviewee,gps_location,...,worker,worker_id,local_muni_clean,geo_local_muni_name,geo_local_muni_code,geo_local_muni_ward_prefix,geo_local_muni_district_code,geo_local_muni_province_code,geo_local_muni_province_number,geo_ward_num_clean
0,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007


In [205]:
# df['geo_ward_num_clean'] = df['geo_ward_num_clean'].astype(str)
local_muni_df = df[
    (df.complete == 'Yes') &
    (df.geo_ward_num_clean.notnull()) &
    (df.geo_local_muni_code.isnull()) &
    ~(df.geo_ward_num_clean.astype(str).str.contains('na'))]
# local_muni_df.loc[:,('local_muni_clean')]
local_muni_df.loc[:,('local_muni_clean')] = map(get_local_muni_name, local_muni_df.loc[:,('geo_ward_num_clean')])

Ekurhuleni
Ethekwini
Ethekwini
Ethekwini
Albert Luthuli
Bushbuckridge
Bushbuckridge
City of Matlosana
City of Matlosana
City of Matlosana
Bitou
Bitou
Bitou
Bitou
Bitou
Bitou


In [207]:
test_df = local_muni_df.copy()

In [208]:
cols = [u'geo_local_muni_name', u'geo_local_muni_code',
       u'geo_local_muni_ward_prefix', u'geo_local_muni_district_code',
       u'geo_local_muni_province_code', u'geo_local_muni_province_number']

for col in cols:
    del local_muni_df[col]
local_muni_df = pd.merge(local_muni_df, lm_info_df, how='left', on="local_muni_clean")

In [ ]:
test_df.loc[df.ID.isin(local_muni_df.ID), cols] = local_muni_df[cols]

In [168]:
df.head(n=2)

,province,ID,date,start,end,started,complete,signature,signature_interviewee,gps_location,...,worker,worker_id,local_muni_clean,geo_local_muni_name,geo_local_muni_code,geo_local_muni_ward_prefix,geo_local_muni_district_code,geo_local_muni_province_code,geo_local_muni_province_number,geo_ward_num_clean
0,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007


In [211]:
test_df

,province,ID,date,start,end,started,complete,signature,signature_interviewee,gps_location,...,worker,worker_id,local_muni_clean,geo_local_muni_name,geo_local_muni_code,geo_local_muni_ward_prefix,geo_local_muni_district_code,geo_local_muni_province_code,geo_local_muni_province_number,geo_ward_num_clean
8486,GT,bea631f6-3e90-4f08-b070-65b22c226326,2014-03-31,2014-03-31 13:16:00 +0200,2014-03-31 13:26:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-26.171552,28.161673",...,Ekhuruleni 2 ~ Ntombizodwa Msibi - 12:52:24 PM...,403c1390-81e7-11e3-966f-001e673400ac,ekurhuleni,NaN,NaN,NaN,NaN,NaN,NaN,79700092
11030,KZN,86e16912-7288-4abe-9f36-9bdb4f396cb7,2014-05-21,2014-05-21 09:12:00 +0200,2014-05-27 15:22:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-29.697030,31.029527",...,NB. Gugulethu Bhengu - 2:25:10 PM 30/5/2014 - ...,7c60d414-c15c-11e3-ac28-001e6733fe3c,ethekwini,NaN,NaN,NaN,NaN,NaN,NaN,59500050
11031,KZN,fdf9884e-09f4-463f-9f8f-b7c864159781,2014-05-26,2014-05-26 13:42:00 +0200,2014-05-27 15:33:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-29.677255,31.020398",...,NB. Gugulethu Bhengu - 2:25:10 PM 30/5/2014 - ...,7c60d414-c15c-11e3-ac28-001e6733fe3c,ethekwini,NaN,NaN,NaN,NaN,NaN,NaN,59500051
11033,KZN,5469dd13-35be-4603-8e8d-abd88eda1b45,2014-05-20,2014-05-20 10:02:00 +0200,2014-05-27 15:18:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-29.909540,31.020890",...,NB. Gugulethu Bhengu - 2:25:10 PM 30/5/2014 - ...,7c60d414-c15c-11e3-ac28-001e6733fe3c,None,NaN,NaN,NaN,NaN,NaN,NaN,59500066
11034,KZN,c01a2a39-391b-4317-8791-637669fdca05,2014-05-23,2014-05-23 12:27:00 +0200,2014-05-23 17:41:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-29.881843,30.911903",...,NB. Rooku K - 9:54:06 AM 4/6/2014 - Logged in ...,cabaae2e-dc0e-11e3-8509-001e6733fe3c,ethekwini,NaN,NaN,NaN,NaN,NaN,NaN,59500063
16393,MP,5336f2c0-bc36-4f9f-883a-b42157dbfefb,2014-03-12,2014-03-12 11:09:00 +0200,2014-04-18 15:28:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-26.046355,30.779757",...,Albert Luthuli. Beauty Nkosi - 9:26:29 AM 9...,5badb7f8-9e09-11e3-bbfd-001e673400ac,albert luthuli,NaN,NaN,NaN,NaN,NaN,NaN,83001024
16450,MP,32b6f764-b002-4d67-8d5b-9ff77c4250eb,2014-03-24,2014-03-24 09:07:26 +0200,2014-03-24 09:41:45 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-24.773478,31.241988",...,Enhlazeni Petronella Glorry Mkhabela - 7:31:46...,b37a8c88-9d6c-11e3-b3ce-001e673400ac,bushbuckridge,NaN,NaN,NaN,NaN,NaN,NaN,83205029
16474,MP,72406869-1418-42c2-872d-6e14462400bb,2014-03-25,2014-03-25 12:07:47 +0200,2014-03-28 20:00:43 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-24.619508,31.055680",...,Enhlazeni Lolicky Pretty Ngobeni - 7:50:40 PM ...,d49f9fa0-9d6e-11e3-b3ce-001e673400ac,bushbuckridge,NaN,NaN,NaN,NaN,NaN,NaN,83205018
18137,NW,d2595f89-0ec9-4497-9e2e-0cb15e6ba5af,2014-04-09,2014-04-09 14:02:00 +0200,2014-04-09 14:16:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-26.892013,26.599790",...,North West Kabelo Raputsoane - 8:44:32 AM 2/5/...,5d72ba0a-baf8-11e3-b6f3-001e6733fe3c,city of matlosana,NaN,NaN,NaN,NaN,NaN,NaN,64003010
18138,NW,b8a7c607-401d-4a47-b365-6c3d627799ce,2014-04-24,2014-04-24 11:53:00 +0200,2014-05-02 08:44:00 +0200,Yes,Yes,https://run.journeyapps.com/media/W1siZiIsIjUz...,https://run.journeyapps.com/media/W1siZiIsIjUz...,"-26.884335,26.598318",...,North West Kabelo Raputsoane - 8:44:32 AM 2/5/...,5d72ba0a-baf8-11e3-b

In [178]:
# df[(df.complete == 'Yes') &
#     (df.geo_ward_num_clean.notnull()) &
#     (df.geo_local_muni_code.isnull()) &
#     (~df.geo_ward_num_clean.str.contains('na'))].loc[:,(cols)] = local_muni_df.loc[:,(cols)]


df.loc[df.ID.isin(local_muni_df.ID), cols] = local_muni_df[cols]
# df.loc[df.Name.isin(df1.Name), ['Nonprofit', 'Education']] = df1[['Nonprofit', 'Education']]
# df.loc[df.ID.isin(local_muni_df.ID), cols]
# local_muni_df.loc[:,(['ID'] + cols)]

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21877 entries, 0 to 21876
Columns: 535 entries, province to geo_ward_num_clean
dtypes: float64(142), int64(151), object(242)
memory usage: 89.5+ MB


In [181]:
df.merge(local_muni_df[cols])

,province,ID,date,start,end,started,complete,signature,signature_interviewee,gps_location,...,worker,worker_id,local_muni_clean,geo_local_muni_name,geo_local_muni_code,geo_local_muni_ward_prefix,geo_local_muni_district_code,geo_local_muni_province_code,geo_local_muni_province_number,geo_ward_num_clean
0,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
2,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
3,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
4,EC,0fa3c7a7-175e-4580-b57d-786709954a89,2013-10-17,2013-10-17 19:58:41 +0200,NaN,Yes,No,NaN,NaN,NaN,...,ITEC Sicelo Dyubele - 4:11:47 PM 26/3/2014 - C...,982fb966-3260-11e3-9f0e-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
5,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007
6,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007
7,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007
8,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007
9,EC,ac670c3c-e6ff-4b00-920c-f96263566e43,2013-11-06,2013-11-06 08:50:45 +0200,NaN,Yes,No,NaN,NaN,"-31.603326,28.791947",...,Masikhule Salathiso Ngumbela - 2:09:51 PM 29/...,3f53a0c2-4602-11e3-b1c4-001e673400ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21507007


In [182]:
# df.loc[df['ID'] == 'bea631f6-3e90-4f08-b070-65b22c226326']
df.columns

Index([u'province', u'ID', u'date', u'start', u'end', u'started', u'complete',
       u'signature', u'signature_interviewee', u'gps_location',
       ...
       u'worker', u'worker_id', u'local_muni_clean', u'geo_local_muni_name',
       u'geo_local_muni_code', u'geo_local_muni_ward_prefix',
       u'geo_local_muni_district_code', u'geo_local_muni_province_code',
       u'geo_local_muni_province_number', u'geo_ward_num_clean'],
      dtype='object', length=535)

In [161]:
df['geo_ward_num_clean'] = df['geo_ward_num_clean'].astype(str)
df[(df.complete == 'Yes') &
    (df.geo_ward_num_clean.notnull()) &
    (df.geo_local_muni_code.isnull()) &
    (~df.geo_ward_num_clean.str.contains('na'))].loc[:,(cols)]

,geo_local_muni_name,geo_local_muni_code,geo_local_muni_ward_prefix,geo_local_muni_district_code,geo_local_muni_province_code,geo_local_muni_province_number


In [59]:
# These entries have no ward number or GPS location associated with them.

df[
    (df.complete == 'Yes') &
    (df.geo_ward_num_clean.isnull()) &
    (df.address_ward_number.isnull()) &
    (df.gps_location.isnull())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 535 entries, province to geo_ward_num_clean
dtypes: float64(142), int64(151), object(242)
memory usage: 0.0+ bytes


In [ ]:
df.to_csv('.csv', index=False)